In [152]:
# Imports
import pandas as pd

tz_info = 'Europe/Madrid'

In [153]:
df = pd.read_excel('../data/Generalitat Extracción_2018.xlsx', skiprows=2)

In [154]:
df.head(10)

,CUPS,EG2,EG3,Fecha inicio Docu. cálculo,Fecha fin Docu. cálculo,Tarifa,Tipo Lectura,Indicadores,Consumo kWh ATR,Consumo kWh GLP,Consumo Kg
0,ES0217010002161818HX,BARCELONA,BADALONA,2017-12-16,2017-12-31,3.4 GAS,ESTIMADA,NaN,7237,0,0.0
1,ES0217010002161818HX,BARCELONA,BADALONA,2018-01-01,2018-01-12,3.4 GAS,REAL,NaN,63714,0,0.0
2,ES0217010002161818HX,BARCELONA,BADALONA,2018-01-13,2018-02-13,3.4 GAS,REAL,NaN,90071,0,0.0
3,ES0217010002161818HX,BARCELONA,BADALONA,2018-02-14,2018-03-13,3.4 GAS,REAL,NaN,74947,0,0.0
4,ES0217010002161818HX,BARCELONA,BADALONA,2018-03-14,2018-04-12,3.4 GAS,REAL,NaN,67338,0,0.0
5,ES0217010002161818HX,BARCELONA,BADALONA,2018-04-13,2018-05-14,3.4 GAS,REAL,NaN,42883,0,0.0
6,ES0217010002161818HX,BARCELONA,BADALONA,2018-05-15,2018-06-13,3.4 GAS,REAL,NaN,24175,0,0.0
7,ES0217010002161818HX,BARCELONA,BADALONA,2018-06-14,2018-07-13,3.4 GAS,REAL,NaN,18104,0,0.0
8,ES0217010002161818HX,BARCELONA,BADALONA,2018-07-14,2018-08-09,3.4 GAS,REAL,NaN,14070,0,0.0
9,ES0217010002161818HX,BARCELONA,BADALONA,2018-08-10,2018-09-17,3.4 GAS,REAL,NaN,21864,0,0.0


In [155]:
# Transform Raw Data

df['Fecha fin Docu. cálculo'] += pd.Timedelta(hours=23)

df['Fecha inicio Docu. cálculo'] = df['Fecha inicio Docu. cálculo'].dt.tz_localize(tz_info)
df['Fecha fin Docu. cálculo'] = df['Fecha fin Docu. cálculo'].dt.tz_localize(tz_info)

# datatime64 [ns] to unix time
df['measurementStart'] = df['Fecha inicio Docu. cálculo'].astype('int') / 10 ** 9
df['measurementStart'] = df['measurementStart'].astype('int')

df['measurementEnd'] = df['Fecha fin Docu. cálculo'].astype('int') / 10 ** 9
df['measurementEnd'] = df['measurementEnd'].astype('int')
df['ts'] = df['measurementStart']

# Calculate kWh
df['measurementValue'] = df['Consumo kWh ATR'].fillna(0) + df['Consumo kWh GLP'].fillna(0)

df = df[['CUPS', 'ts', 'measurementStart', 'measurementEnd', 'measurementValue', 'Tipo Lectura']]

In [198]:
# Group by CUPS
for cups, sub_df in df.groupby('CUPS'):
    sub_df.sort_values(by=['measurementStart'], inplace=True)

    # https://towardsdatascience.com/efficiently-iterating-over-rows-in-a-pandas-dataframe-7dd5f9992c01

    # Shift
    sub_df['measurementStart_shifted'] = sub_df['measurementStart'].shift(-1)
    sub_df['measurementStart_shifted'] = sub_df['measurementStart_shifted'].astype('Int64')

    # Compare invoices start and end dates
    sub_df['is_gap'] = (sub_df['measurementStart_shifted'] - sub_df['measurementEnd']).to_list()

    # Find gaps between invoices
    sub_df.loc[sub_df["is_gap"] != 3600, "is_gap"] = True
    sub_df.loc[sub_df["is_gap"] == 3600, "is_gap"] = False
    sub_df.loc[sub_df["measurementStart_shifted"].isnull(), "is_gap"] = False

    # Split Dataframes
    list_of_dataframes = []
    if True in sub_df['is_gap'].unique():

        last_status = None
        index = None

        for row in sub_df.itertuples():

            if last_status is None:
                index = 0
                list_of_dataframes.append([])

            if last_status is not None and row.is_gap is True:
                index += 1
                list_of_dataframes.append([])

            if last_status is not None and row.is_gap is False and last_status is True:
                index += 1
                list_of_dataframes.append([])

            list_of_dataframes[index].append(row)
            last_status = row.is_gap

        for i in range(len(list_of_dataframes)):
            new_df = pd.DataFrame(list_of_dataframes[i])
            new_df.rename(columns={'_6': 'Tipo Lectura'}, inplace=True)
            list_of_dataframes[i] = new_df

    else:
        list_of_dataframes.append(sub_df)

    for df_i in list_of_dataframes:
        if 'REAL' in df_i['Tipo Lectura'].unique():
            # Drop n rows until tipo lectura = REAL
            first_real = df_i[df_i['Tipo Lectura'] == 'REAL'].index[0]

            df_i = df_i.loc[first_real:]

        df_i['cumsum'] = df_i['measurementValue'].cumsum()

                    CUPS          ts  measurementStart  measurementEnd  \
1   ES0217010002161818HX  1514761200        1514761200      1515794400   
2   ES0217010002161818HX  1515798000        1515798000      1518559200   
3   ES0217010002161818HX  1518562800        1518562800      1520978400   
4   ES0217010002161818HX  1520982000        1520982000      1523566800   
5   ES0217010002161818HX  1523570400        1523570400      1526331600   
6   ES0217010002161818HX  1526335200        1526335200      1528923600   
7   ES0217010002161818HX  1528927200        1528927200      1531515600   
8   ES0217010002161818HX  1531519200        1531519200      1533848400   
9   ES0217010002161818HX  1533852000        1533852000      1537218000   
10  ES0217010002161818HX  1537221600        1537221600      1539032400   
11  ES0217010002161818HX  1539036000        1539036000      1541628000   
12  ES0217010002161818HX  1541631600        1541631600      1544565600   

    measurementValue Tipo Lectura  me